# Session Demo
##### Supercharging Machine Learning Tasks with Ray: An Open-Source Unified Compute Framework


Creation by: Supasate Vorathammathorn

## Prerequisite

In [1]:
!nvidia-smi

Fri Dec 15 16:11:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!pip install tensorflow==2.15.0
!pip install torch
!pip install ray
!pip install qiskit
!pip install qiskit-machine-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 66.6 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 66.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.9/306.9 kB 32.2 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-py3-none-any.whl size=3566 sha256=a1fce5e41592643803a50bd27dc5f98b2fd5d0e76c541f40f0736fc0011dbf61
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


## MNIST Dataset

#### Traditional Machine Learning

In [36]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import time

# Force TensorFlow to use only the CPU
os.environ['CUDA_VISIBLE_DEVICES'] = ''

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Define a simple neural network model
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Traditional sequential training and Timer
start_time = time.time()

model = create_model()
model.fit(x_train, y_train, epochs=100)
results = model.evaluate(x_test, y_test)

end_time = time.time()
print("\nTraditional Sequential Results:", results)
print("Time taken:", end_time - start_time, "seconds")

Epoch 1/100
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2613 - accuracy: 0.9258
Epoch 2/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1138 - accuracy: 0.9666
Epoch 3/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0777 - accuracy: 0.9775
Epoch 4/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0586 - accuracy: 0.9822
Epoch 5/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0451 - accuracy: 0.9859
Epoch 6/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0362 - accuracy: 0.9890
Epoch 7/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0276 - accuracy: 0.9914
Epoch 8/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0232 - accuracy: 0.9930
Epoch 9/100
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0189 - accuracy: 0.9942
Epoch 10/100
1875/1875 [==============================] - 3s 2ms/step - l

#### Parallel Computing

In [1]:
import multiprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import time

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Define a simple neural network model
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define training config
def train_model(data):
    (x_subset, y_subset), model = data
    model.fit(x_subset, y_subset, epochs=100, verbose=False)
    return model.evaluate(x_test, y_test)

# Parallel Computing and Timer
num_processors = multiprocessing.cpu_count()
data_chunks = (np.array_split(x_train, num_processors), np.array_split(y_train, num_processors))

start_time = time.time()

processes=num_processors
pool = multiprocessing.Pool(processes=num_processors)
print(processes)
results = pool.map(train_model, [(d, create_model()) for d in zip(*data_chunks)])
pool.close()
pool.join()

end_time = time.time()

print("\nParallel Computing Results:", results)
print("Time taken:", end_time - start_time, "seconds")

2023-12-15 18:37:30.883292: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 18:37:30.886640: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 18:37:30.929674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 18:37:30.929712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 18:37:30.930825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

16


2023-12-15 18:37:34.405003: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 18:37:34.405547: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-15 18:37:35.323480: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/li

313/313 [==============================] - 1s 1ms/step - loss: 0.4872 - accuracy: 0.9306

Parallel Computing Results: [[0.3777349293231964, 0.9395999908447266], [0.47800907492637634, 0.9314000010490417], [0.40980035066604614, 0.9377999901771545], [0.3238486349582672, 0.9470999836921692], [0.4099157154560089, 0.9398000240325928], [0.40331515669822693, 0.9366999864578247], [0.449887752532959, 0.9362000226974487], [0.43649110198020935, 0.9361000061035156], [0.3985280394554138, 0.9416000247001648], [0.4585290551185608, 0.9316999912261963], [0.43525072932243347, 0.9387000203132629], [0.39306217432022095, 0.9401000142097473], [0.41321054100990295, 0.9358999729156494], [0.38110873103141785, 0.9383999705314636], [0.41022977232933044, 0.935699999332428], [0.48716068267822266, 0.9305999875068665]]
Time taken: 87.71605038642883 seconds


#### Distributed Computing with Ray Framework

In [2]:
import ray
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
import time

ray.init()

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Define a simple neural network model
@ray.remote
class MNISTModel:
    def __init__(self):
        self.model = Sequential([
            Flatten(input_shape=(28, 28)),
            Dense(128, activation='relu'),
            Dense(10, activation='softmax')
        ])
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, x_subset, y_subset):
        self.model.fit(x_subset, y_subset, epochs=100, verbose=False)
        return self.model.evaluate(x_test, y_test)

# Split dataset for distributed processing
num_chunks = 16  # Adjust based on your cluster setup
data_chunks = (np.array_split(x_train, num_chunks), np.array_split(y_train, num_chunks))

# Measure the time for distributed model training
start_time = time.time()

models = [MNISTModel.remote() for _ in range(num_chunks)]
result_ids = [models[i].train.remote(x, y) for i, (x, y) in enumerate(zip(*data_chunks))]
results = ray.get(result_ids)

end_time = time.time()
ray.shutdown()

print("Ray Framework Results:", results)
print("Time taken:", end_time - start_time, "seconds")

2023-12-15 18:39:29,574	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8326299648 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-12-15 18:39:29,718	INFO worker.py:1673 -- Started a local Ray instance.
2023-12-15 18:39:31,285	WARNING worker.py:2074 -- Warning: The actor MNISTModel is very large (60 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(pid=10580) 2023-12-15 18:39:33.328098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factor

313/313 [==============================] - 1s 2ms/step - loss: 0.3601 - accuracy: 0.9436


(pid=10587) 2023-12-15 18:39:34.234719: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered [repeated 15x across cluster]
(pid=10587) 2023-12-15 18:39:34.234773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered [repeated 15x across cluster]
(pid=10587) 2023-12-15 18:39:34.236395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered [repeated 15x across cluster]
(pid=10587) 2023-12-15 18:39:35.660438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT [repeated 15x across cluster]
(MNISTModel pid=10587) 2023-12-15 18:39:38.142601: E external/local_xla/x

Ray Framework Results: [[0.4179271459579468, 0.9380000233650208], [0.4578479528427124, 0.9307000041007996], [0.39505597949028015, 0.9376000165939331], [0.36012786626815796, 0.9435999989509583], [0.3953828513622284, 0.9409999847412109], [0.3922625482082367, 0.9366999864578247], [0.4776998460292816, 0.9358999729156494], [0.42379653453826904, 0.9359999895095825], [0.383792906999588, 0.9423999786376953], [0.46945318579673767, 0.9304999709129333], [0.42608609795570374, 0.9376999735832214], [0.38524702191352844, 0.9398999810218811], [0.4165307581424713, 0.9347000122070312], [0.41118794679641724, 0.9381999969482422], [0.39745181798934937, 0.9359999895095825], [0.45669063925743103, 0.9302999973297119]]
Time taken: 93.40018606185913 seconds


## Iris Dataset

#### Traditional Machine Learning

In [5]:
import os
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

# Force TensorFlow to use only the CPU
os.environ['CUDA_VISIBLE_DEVICES'] = ''

# Load Iris data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One hot encoding of the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network model for the Iris dataset
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')  # 3 output units for 3 classes of Iris
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Traditional sequential training and Timer
start_time = time.time()

model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=100, verbose=False)
results = model.evaluate(X_test, y_test)

end_time = time.time()
print("\nTraditional Sequential Results:", results)
print("Time taken:", end_time - start_time, "seconds")


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


1/1 [==============================] - 0s 112ms/step - loss: 0.0985 - accuracy: 0.9667

Traditional Sequential Results: [0.09852643311023712, 0.9666666388511658]
Time taken: 1.3401374816894531 seconds


#### Parallel Computing

In [1]:
import multiprocessing
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

# Load Iris data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One hot encoding of the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network model for the Iris dataset
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')  # 3 output units for 3 classes of Iris
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def train_model(data):
    (x_subset, y_subset), model = data
    model.fit(x_subset, y_subset, epochs=100, verbose=False)
    return model.evaluate(X_test, y_test)

num_processors = multiprocessing.cpu_count()
print(num_processors)
data_chunks = (np.array_split(X_train, num_processors), np.array_split(y_train, num_processors))

start_time = time.time()

pool = multiprocessing.Pool(processes=num_processors)
results = pool.map(train_model, [(d, create_model(X_train.shape[1])) for d in zip(*data_chunks)])
pool.close()
pool.join()

end_time = time.time()

print("\nParallel Computing Results:", results)
print("Time taken:", end_time - start_time, "seconds")

2023-12-15 20:15:54.325187: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 20:15:54.328588: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 20:15:54.371554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 20:15:54.371595: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 20:15:54.372737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

16


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-12-15 20:15:57.875358: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 20:15:57.875900: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2

1/1 [==============================] - 0s 137ms/step - loss: 0.2665 - accuracy: 0.9000

Parallel Computing Results: [[0.37819844484329224, 0.8666666746139526], [0.4738393723964691, 0.6333333253860474], [0.40992823243141174, 0.8999999761581421], [0.3147142827510834, 0.9666666388511658], [0.3522648513317108, 0.8333333134651184], [0.34637659788131714, 1.0], [0.39105910062789917, 0.8333333134651184], [0.32245832681655884, 0.8999999761581421], [0.4221167266368866, 0.699999988079071], [0.3826830983161926, 0.800000011920929], [0.47103989124298096, 0.7333333492279053], [0.47057756781578064, 0.8666666746139526], [0.25190502405166626, 0.8666666746139526], [0.2664952874183655, 0.8999999761581421], [2.1014530658721924, 0.6666666865348816], [0.30188363790512085, 0.8999999761581421]]
Time taken: 4.35579776763916 seconds


In [5]:
#### Distributed Computing with Ray Framework

In [4]:
import ray
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

ray.init()

# Load Iris data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One hot encoding of the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple neural network model for the Iris dataset
@ray.remote
class IrisModel:
    def __init__(self, input_shape):
        self.model = Sequential([
            Dense(64, activation='relu', input_shape=(input_shape,)),
            Dense(64, activation='relu'),
            Dense(3, activation='softmax')  # 3 output units for 3 classes of Iris
        ])
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    def train(self, x_subset, y_subset):
        self.model.fit(x_subset, y_subset, epochs=100, verbose=False)
        return self.model.evaluate(X_test, y_test)

# Split dataset for distributed processing
num_chunks = 16  # Adjust based on your cluster setup
data_chunks = (np.array_split(X_train, num_chunks), np.array_split(y_train, num_chunks))

# Measure time for distributed model training
start_time = time.time()

models = [IrisModel.remote(X_train.shape[1]) for _ in range(num_chunks)]
result_ids = [models[i].train.remote(x, y) for i, (x, y) in enumerate(zip(*data_chunks))]
results = ray.get(result_ids)

end_time = time.time()
ray.shutdown()

print("Ray Framework Results:", results)
print("Time taken:", end_time - start_time, "seconds")

2023-12-15 20:44:09,913	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8326295552 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-12-15 20:44:09,962	INFO worker.py:1673 -- Started a local Ray instance.
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
(pid=16842) 2023-12-15 20:44:12.470649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register

1/1 [==============================] - 0s 236ms/step - loss: 0.2775 - accuracy: 0.9667 [repeated 12x across cluster]


(pid=16854) 2023-12-15 20:44:12.570999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered [repeated 15x across cluster]
(pid=16854) 2023-12-15 20:44:12.571054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered [repeated 15x across cluster]
(pid=16854) 2023-12-15 20:44:12.572628: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered [repeated 15x across cluster]
(pid=16839) 2023-12-15 20:44:13.993326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT [repeated 15x across cluster]
(IrisModel pid=16853) 2023-12-15 20:44:16.626308: E external/local_xla/xl

Ray Framework Results: [[0.2775537073612213, 0.8666666746139526], [0.28768059611320496, 0.8333333134651184], [0.26196369528770447, 0.9666666388511658], [0.27752044796943665, 0.9666666388511658], [0.4339108169078827, 0.800000011920929], [0.31653356552124023, 1.0], [0.31879183650016785, 0.8666666746139526], [0.3247644603252411, 0.8666666746139526], [0.5704891681671143, 0.699999988079071], [0.29171842336654663, 0.8999999761581421], [0.2786290943622589, 0.9666666388511658], [0.24089689552783966, 0.9666666388511658], [0.1996387541294098, 0.9333333373069763], [0.2783876061439514, 0.9666666388511658], [3.035949468612671, 0.6666666865348816], [0.19806383550167084, 0.9666666388511658]]
Time taken: 7.995192766189575 seconds
